In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
from result_saver import SaverProvider

provider = SaverProvider()

In [ ]:
from Scratch import metadata_loader

DEVICE = 'ibm_sherbrooke'
LOGICAL = '0'
XBASIS = False

ROUNDS = 50
RESETS = True


md = metadata_loader(True, True)

md = md[md["job_status"] == "JobStatus.DONE"]
md = md[md["code"] == "RepetitionCodeCircuit"]
# # md = md[md["descr"] == 'subset RepCodes']
md = md.dropna(subset=["rounds"])
md = md[md["meas_level"] == 1]
md['rounds'] = md['rounds'].astype(int)
md['distance'] = md['distance'].astype(int)

md = md[md["backend_name"] == DEVICE]
# md = md[md["logical"] == LOGICAL]
md = md[md["logical"] == int(LOGICAL)]
md = md[md["xbasis"] == XBASIS]
md = md[md["rounds"] == ROUNDS]
md = md[md["resets"] == RESETS]

md = md[md['notebook_name']=='Kyoto_retrieve_jobs.ipynb']


print(md.shape)
md[-1:]
# md

(0, 18)


,creation_date,notebook_name,backend_name,job_id,tags,meas_level,shots,num_qubits,job_status,execution_date,code,distance,rounds,logical,descr,resets,xbasis,path_info


In [ ]:
md_loc = 0

job = provider.retrieve_job(md.iloc[md_loc]["job_id"])

IndexError: single positional indexer is out-of-bounds

In [ ]:
DISTANCE = md.iloc[md_loc]["distance"]
RESETS = False

mmr = job.result().get_memory()

print((DISTANCE-1)*ROUNDS + DISTANCE)
mmr.shape

2500


(1569, 2500)

In [ ]:
import numpy as np

# put a 1 where the real part of mmr is > 0 and 0 otherwise
countMat = np.where(np.real(mmr)>0, 1, 0)

# Print the number of ones vs the number of zeros
print(np.sum(countMat), countMat.size - np.sum(countMat))
print(f"number of 1s: {np.sum(countMat)}")
print(f"number of 0s: {countMat.size - np.sum(countMat)}")
print(f"total number of elements: {countMat.size}")

1818717 2103783
number of 1s: 1818717
number of 0s: 2103783
total number of elements: 3922500


In [ ]:
arr = np.array([-1, -1, -1 , 1])
count = np.where(np.real(arr)>0, 1, 0)

print(count)

[0 0 0 1]


# Decode counts

In [ ]:
import pymatching
import stim
from soft_info import RepetitionCodeStimCircuit

# noise_list = [2e-2, 1e-3, 1e-2, 1e-2, 2e-2, 1e-2, 1e-2]
# noise_list = [1e-2]*7
# code = RepetitionCodeStimCircuit(DISTANCE, ROUNDS, XBASIS, resets=RESETS, noise_list=noise_list)

# stim_circ = code.circuits[LOGICAL]

stim_circ = stim.Circuit.generated("repetition_code:memory",
                                distance=DISTANCE,
                                rounds=ROUNDS,
                                after_clifford_depolarization=1e-2, #two-qubit-fidelity,
                                after_reset_flip_probability=45e-2, #reset error,
                                before_measure_flip_probability=3e-2, #measurement error,
                                before_round_data_depolarization=1e-2) #idle error)


matching = pymatching.Matching.from_detector_error_model(stim_circ.detector_error_model())

In [ ]:
from tqdm import tqdm
from soft_info import counts_to_det_syndr

def countrow_to_count_str(countrow):
    count_str = ''
    for idx, i in enumerate(countrow):
        if i == 1:
            bit = '1'
        else:
            bit = '0'
        
        if (idx+1) % (DISTANCE-1) == 0 and idx <= (DISTANCE-1)*ROUNDS:
            count_str += bit + ' '
        else:
            count_str += bit

    # invert the count string
    count_str = count_str[::-1]

    return count_str

num_errors = 0
count_key_first = []
for countrow in tqdm(countMat):
    count_key = countrow_to_count_str(countrow)

    array_processed_str = counts_to_det_syndr(count_key, _resets=RESETS)

    predicted_observable = matching.decode(array_processed_str)

    count_key_first.append(count_key[0])
    actual_observable = [(int(count_key[0])+int(LOGICAL))%2]
    # actual_observable = [(int(count_key[0])+1)%2]

    if actual_observable != predicted_observable:
        num_errors += 1

state = 'X' if XBASIS else 'Z'
state += f"{LOGICAL} {ROUNDS}"
print(f"\nstate {state}")
print(f"num_errors: {num_errors} out of {countMat.shape[0]}")


  0%|          | 0/1569 [00:00<?, ?it/s]

100%|██████████| 1569/1569 [00:07<00:00, 219.21it/s]


state Z1 50
num_errors: 15 out of 1569


In [ ]:
print(np.sum(countMat[:, -1]))
print(len(countMat))

1501
1569


In [ ]:
from qiskit_qec.circuits import RepetitionCodeCircuit

code = RepetitionCodeCircuit(3, 2, xbasis=XBASIS, resets=RESETS)
qc = code.circuit[LOGICAL]

qc.draw(fold=-1)

┌───┐     ┌───┐     ┌─┐          ┌───┐     ┌───┐        ┌─┐         
      link_qubit_0: ┤ X ├─────┤ X ├─────┤M├─|0>──────┤ X ├─────┤ X ├────────┤M├─────────
                    └─┬─┘┌───┐└─┬─┘┌───┐└╥┘ ┌─┐      └─┬─┘┌───┐└─┬─┘┌───┐   └╥┘┌─┐      
      link_qubit_1: ──┼──┤ X ├──┼──┤ X ├─╫──┤M├──|0>───┼──┤ X ├──┼──┤ X ├────╫─┤M├──────
                      │  └─┬─┘  │  └─┬─┘ ║  └╥┘        │  └─┬─┘  │  └─┬─┘┌─┐ ║ └╥┘      
      code_qubit_0: ──■────┼────┼────┼───╫───╫─────────■────┼────┼────┼──┤M├─╫──╫───────
                           │    │    │   ║   ║              │    │    │  └╥┘ ║  ║ ┌─┐   
      code_qubit_1: ───────■────■────┼───╫───╫──────────────■────■────┼───╫──╫──╫─┤M├───
                                     │   ║   ║                        │   ║  ║  ║ └╥┘┌─┐
      code_qubit_2: ─────────────────■───╫───╫────────────────────────■───╫──╫──╫──╫─┤M├
                                         ║   ║                            ║  ║  ║  ║ └╥┘
round_0_link_bit: 2/═════════════════════╩═══╩════════════════════════════╬══╬══╬══╬══╬═
                                         0   1                            ║  ║  ║  ║  ║ 
round_1_link_bit: 2/══════════════════════════════════════════════════════╬══╩══╩══╬══╬═
                                                                          ║  0  1  ║  ║ 
        code_bit: 3/══════════════════════════════════════════════════════╩════════╩══╩═
                                                                          0        1  2